In [1]:
import pandas as pd
import yfinance as yf
import time
import os
import scraping_tickers

def get_stock_data(tickers_stocks_yf, nom_indice, ticker_indice_yf, file_path):
    
    # Création du DataFrame initial avec une colonne vide
    df = pd.DataFrame({'Ticker_Stocks_Yf': tickers_stocks_yf})

    # Récupérer toutes les données en une seule requête
    tickers_obj = yf.Tickers(" ".join(tickers_stocks_yf))  # Récupération groupée des tickers

    # Initialisation des listes pour stocker les données
    short_name_stocks = []
    pays_stocks = []
    place_boursiere = []
    secteur_activite = []
    capitalisation_boursiere = []
    tickers_stocks = []

    for i in tickers_stocks_yf:
        try:
            # Récupérer les informations via l'objet Tickers
            info = tickers_obj.tickers[i].info

            # Extraction du ticker sans suffixe
            symbole_sans_suffixe = i.split(".")[0]
            tickers_stocks.append(symbole_sans_suffixe)

            # Ajouter les informations dans leurs listes respectives
            short_name_stocks.append(info.get('shortName', 'Non disponible'))
            pays_stocks.append(info.get('country', 'Non disponible'))
            place_boursiere.append(info.get('exchange', 'Non disponible'))
            secteur_activite.append(info.get('sector', 'Non disponible'))
            capitalisation_boursiere.append(info.get('marketCap', None))

        except Exception as e:
            print(f"Erreur lors de la récupération des données pour {i}: {e}")
            short_name_stocks.append('Non disponible')
            pays_stocks.append('Non disponible')
            place_boursiere.append('Non disponible')
            secteur_activite.append('Non disponible')
            capitalisation_boursiere.append(None)
            tickers_stocks.append(i)  # Ajouter le ticker original en cas d'erreur

        time.sleep(0.5)  # Petite pause pour éviter le blocage

    # Ajout des colonnes au DataFrame
    df['Short_Name_Stocks'] = short_name_stocks
    df['Pays_Stocks'] = pays_stocks
    df['Place_Boursiere'] = place_boursiere
    df['Secteur_Activite'] = secteur_activite
    df['Capitalisation_Boursiere'] = capitalisation_boursiere
    df['Ticker_Stocks'] = tickers_stocks
    df['Nom_Indice'] = nom_indice
    df['Nombres_Entreprises'] = len(short_name_stocks)
    df['Ticker_Indice_Yf'] = ticker_indice_yf

    # Calcul de la capitalisation boursière totale
    total_capitalisation = df['Capitalisation_Boursiere'].fillna(0).sum()

    # Calcul pondération
    df['Ponderation'] = round((df['Capitalisation_Boursiere'] / total_capitalisation) * 100, 2)

    # Réorganisation des colonnes
    df = df[
        ['Short_Name_Stocks', 'Ticker_Stocks_Yf', 'Ticker_Stocks', 'Secteur_Activite', 'Pays_Stocks', 'Place_Boursiere', 'Capitalisation_Boursiere',
         'Nom_Indice', 'Ticker_Indice_Yf', 'Nombres_Entreprises', 'Ponderation']
    ]

    # Exportation en CSV
    df.to_csv(file_path, index=False, encoding='utf-8')
    print(f"[✅] Le fichier pour l'indice '{nom_indice}' a bien été enregistré sous le nom : {file_path}")

    return df


def csv_indices(dossier_csv):
    
    # Récupérer les tickers via scraping_tickers.all_tickers_yf()
    tickers_yf = scraping_tickers.all_tickers_yf()
    
    # Appels avec jointure automatique du chemin
    df_cac40 = get_stock_data(tickers_yf["CAC40"], "CAC 40", "^FCHI", os.path.join(dossier_csv, "composition_france.csv"))
'''    df_dax40 = get_stock_data(tickers_yf["DAX40"], "DAX", "^GDAXI", os.path.join(base_csv_path, "composition_dax.csv"))
    df_italie40 = get_stock_data(tickers_yf["Italie40"], "FTSE MIB Index", "FTSEMIB.MI", os.path.join(base_csv_path, "composition_italie.csv"))
    df_espagne35 = get_stock_data(tickers_yf["Espagne35"], "IBEX 35", "^IBEX", os.path.join(base_csv_path, "composition_espagne.csv"))
    df_angleterre = get_stock_data(tickers_yf["Angleterre100"], "FTSE 100", "^FTSE", os.path.join(base_csv_path, "composition_angleterre.csv"))
    df_nasdaq100 = get_stock_data(tickers_yf["NASDAQ100"], "NASDAQ 100", "^NDX", os.path.join(base_csv_path, "composition_nasdaq100.csv"))
    df_dowjones = get_stock_data(tickers_yf["Dow Jones"], "Dow Jones 30", "^DJI", os.path.join(base_csv_path, "composition_dowjones.csv"))
    df_belgique = get_stock_data(tickers_yf["Belgique20"], "BEL 20", "^BFX", os.path.join(base_csv_path, "composition_belgique.csv"))
    df_pays_bas = get_stock_data(tickers_yf["Paysbas25"], "AEX-Index", "^AEX", os.path.join(base_csv_path, "composition_paysbas.csv"))
    df_finlande = get_stock_data(tickers_yf["Finlande25"], "OMX Helsinki 25", "^OMXH25", os.path.join(base_csv_path, "composition_finlande.csv"))
    df_suede = get_stock_data(tickers_yf["Suède30"], "OMX Stockholm 30", "^OMXS30", os.path.join(base_csv_path, "composition_suede.csv"))
    df_danemark = get_stock_data(tickers_yf["Danemark25"], "OMX Copenhagen 25", "^OMXC25", os.path.join(base_csv_path, "composition_danemark.csv"))
    df_stoxx50 = get _stock_data(tickers_yf["STOXX50"], "STOXX 50", "^STOXX50E", os.path.join(base_csv_path, "composition_europe50.csv"))
    df_sp500 = get_stock_data(tickers_yf["SP500"], "S&P 500", "^GSPC", os.path.join(base_csv_path, "composition_sp500.csv"))
    df_japon225 = get_stock_data(tickers_yf["Japon225"], "Nikkei 225", "^N225", os.path.join(base_csv_path, "composition_japon.csv"))

    '''
if __name__ == "__main__":
    indices_csv = csv_indices(dossier_csv = "csv") #Appel de la fonction 

[✅] Le fichier scraping a bien été enregistré
Erreur lors de la récupération des données pour MC.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour RMS.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour OR.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour AIR.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour SU.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour EL.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour SAF.PA: Too Many Requests. Rate limited. Try after a while.
Erreur lors de la récupération des données pour TTE.PA: Too Many Requests. Rate limited. Try after a while.


KeyboardInterrupt: 

In [4]:
pip install --upgrade yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.55
    Uninstalling yfinance-0.2.55:
      Successfully uninstalled yfinance-0.2.55
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pygame
import sys

# --- Initialisation ---
pygame.init()

# Taille de la fenêtre
WIDTH, HEIGHT = 600, 400
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mini Pac-Man")

# Couleurs
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)

# --- Joueur ---
player_size = 20
player_x, player_y = WIDTH // 2, HEIGHT // 2
player_speed = 5

# --- Points à collecter ---
points = [(50, 50), (550, 50), (50, 350), (550, 350)]
point_radius = 8

# --- Score ---
score = 0
font = pygame.font.SysFont(None, 30)

# --- Boucle principale ---
clock = pygame.time.Clock()
running = True
while running:
    clock.tick(30)
    win.fill(BLACK)
    
    # --- Événements ---
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    
    # --- Déplacements ---
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_x -= player_speed
    if keys[pygame.K_RIGHT]:
        player_x += player_speed
    if keys[pygame.K_UP]:
        player_y -= player_speed
    if keys[pygame.K_DOWN]:
        player_y += player_speed
    
    # --- Limites fenêtre ---
    player_x = max(0, min(WIDTH - player_size, player_x))
    player_y = max(0, min(HEIGHT - player_size, player_y))
    
    # --- Dessiner joueur ---
    pygame.draw.circle(win, YELLOW, (player_x + player_size // 2, player_y + player_size // 2), player_size // 2)
    
    # --- Dessiner points ---
    for point in points[:]:
        pygame.draw.circle(win, BLUE, point, point_radius)
        # Collision joueur/point
        px, py = point
        if abs(player_x + player_size//2 - px) < player_size and abs(player_y + player_size//2 - py) < player_size:
            points.remove(point)
            score += 1
    
    # --- Afficher score ---
    text = font.render(f"Score: {score}", True, WHITE)
    win.blit(text, (10, 10))
    
    pygame.display.update()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
